Part 1  Built Dataframe
****************

#!conda install -c conda-forge folium #install at ibm notebook

In [1]:
import requests
import pandas as pd 
import numpy as np 
import folium

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
df = pd.read_html(url)

In [4]:
ndf = pd.DataFrame(df[0])

Note: Replace 'Not assigned' to 'np.nan' by numpy value and then using 'dropna' for cleaning 'np.nan' values

In [5]:
ndf.replace('Not assigned', np.nan, inplace= True)

In [6]:
ndf.dropna(inplace=True)

In [7]:
ndf.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
ndf.shape

(103, 3)

***

Part 2 Geocoding
****************

In [9]:
coordinate = pd.read_csv('https://cocl.us/Geospatial_data')
coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
geocode = ndf.merge(coordinate, on='Postal Code')
geocode.rename(columns={"Postal Code": "PosCode"}, inplace=True)
geocode.head()

,PosCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


***

Part 3 Generate Maps
****************

Note: Filter 'Toronto' only from Borough column and using Folium for map

In [11]:
toronto = geocode[geocode['Borough'].str.contains("Toronto")]
toronto.head()

,PosCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [12]:
toronto_map = folium.Map(location=[43.651070,-79.347015],zoom_start=11.5)

In [13]:
toronto_place = folium.map.FeatureGroup()

In [14]:
for lat, lng, in zip(toronto.Latitude, toronto.Longitude):
    toronto_place.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=3,
            color='blue',
            fill=True,
            fill_color='yellow',
            fill_opacity=0.6
        )
    )
latitudes = list(toronto.Latitude)
longitudes = list(toronto.Longitude)
labels = list(toronto.Neighborhood)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(toronto_place)    

toronto_map.add_child(toronto_place)